<a href="https://colab.research.google.com/github/Atiwat-R/bias-ai/blob/main/bias_ai.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers torch

  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl.metadata (1.8 kB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinu

In [2]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import sys
from tqdm.notebook import tqdm
import torch

print('System Version:', sys.version)
print('PyTorch version', torch.__version__)
print('Numpy version', np.__version__)
print('Pandas version', pd.__version__)

System Version: 3.10.12 (main, Jul 29 2024, 16:56:48) [GCC 11.4.0]
PyTorch version 2.3.1+cu121
Numpy version 1.26.4
Pandas version 2.1.4


## Step: Load Dataset

In [3]:
import pandas as pd

# Replace 'your_file.csv' with the name of your uploaded file
df = pd.read_csv('/content/mbic.csv')

# Display the first few rows of the DataFrame
df[["text", "label"]]

,text,label
0,The transgender effort to suppress any recogni...,Non-biased
1,Radical Virginia Citizens Defense League has o...,Non-biased
2,Miller is the architect of President Donald Tr...,Non-biased
3,"The House Democrats’ 1,400-page coronavirus re...",Non-biased
4,A specter is haunting the West; our elites see...,Non-biased
...,...,...
17770,Oregon first graders could attend gun safety c...,Biased
17771,But President Donald Trump and his campaign ad...,Biased
17772,The normalization effort included softened rhe...,Biased
17773,"And so, while demonstrations began to spread a...",Biased


In [4]:
texts = df["text"].tolist()  # List of sentences
labels = df["label"].tolist()  # Corresponding labels

## Step: Prepare the Data

In [5]:
# Split data into train, test, validation
from sklearn.model_selection import train_test_split

# Split data into training and test sets
train_texts, test_texts, train_labels, test_labels = train_test_split(texts, labels, test_size=0.2, random_state=42)

# Further split the training data into training and validation sets
train_texts, val_texts, train_labels, val_labels = train_test_split(train_texts, train_labels, test_size=0.25, random_state=42)
# 0.25 * 0.8 = 0.2 of the original data goes to validation

In [6]:
# Encode labels into numbers
from sklearn.preprocessing import LabelEncoder

# Convert labels to integers
label_encoder = LabelEncoder()
label_encoder.fit(labels)  # Fit on the entire set of labels

# Transform the labels for each dataset split
encoded_train_labels = label_encoder.transform(train_labels)
encoded_val_labels = label_encoder.transform(val_labels)
encoded_test_labels = label_encoder.transform(test_labels)

In [7]:
# Tokenize the texts
from transformers import DistilBertTokenizer

# Load the pre-trained BERT tokenizer
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')

# Tokenize each texts set
train_encodings = tokenizer(train_texts, truncation=True, padding=True, max_length=512, return_tensors='pt')
val_encodings = tokenizer(val_texts, truncation=True, padding=True, max_length=512, return_tensors='pt')
test_encodings = tokenizer(test_texts, truncation=True, padding=True, max_length=512, return_tensors='pt')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

In [8]:
# Design Dataset to keep data
from torch.utils.data import Dataset

class TextDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

In [9]:
train_dataset = TextDataset(train_encodings, encoded_train_labels)
val_dataset = TextDataset(val_encodings, encoded_val_labels)
test_dataset = TextDataset(test_encodings, encoded_test_labels)

## Step: Training the Model

In [10]:
# Load the pre-trained model
from transformers import DistilBertForSequenceClassification

# Load the BERT model for sequence classification
# model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)
model = DistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased', num_labels=len(label_encoder.classes_))

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [11]:
from transformers import Trainer, TrainingArguments

# Define training arguments
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=7,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    warmup_steps=1000,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
    eval_strategy="epoch",
    learning_rate=3e-5,
    lr_scheduler_type='cosine'
)

# Initialize the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset
)

In [12]:
# Train the model
trainer.train()

<ipython-input-8-bd375230bbef>:10: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}


Epoch,Training Loss,Validation Loss
1,0.583100,0.623411
2,0.604900,0.583421
3,0.602600,0.591739
4,0.555600,0.608774
5,0.556100,0.599781
6,0.536300,0.629342


<ipython-input-8-bd375230bbef>:10: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
<ipython-input-8-bd375230bbef>:10: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
<ipython-input-8-bd375230bbef>:10: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
<ipython-input-8-bd375230bbef>:10: UserWarning: To copy construct from a 

Epoch,Training Loss,Validation Loss
1,0.583100,0.623411
2,0.604900,0.583421
3,0.602600,0.591739
4,0.555600,0.608774
5,0.556100,0.599781
6,0.536300,0.629342
7,0.485500,0.645424


<ipython-input-8-bd375230bbef>:10: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}


TrainOutput(global_step=2338, training_loss=0.5602511354141301, metrics={'train_runtime': 777.6705, 'train_samples_per_second': 95.998, 'train_steps_per_second': 3.006, 'total_flos': 2298502117095660.0, 'train_loss': 0.5602511354141301, 'epoch': 7.0})

In [13]:
eval_results = trainer.evaluate(test_dataset)
print(eval_results)

<ipython-input-8-bd375230bbef>:10: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}


{'eval_loss': 0.6541956663131714, 'eval_runtime': 10.8243, 'eval_samples_per_second': 328.427, 'eval_steps_per_second': 10.347, 'epoch': 7.0}


In [14]:
# Prediction

def predict(text):
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

    # Tokenize the new data
    new_encodings = tokenizer(text, truncation=True, padding=True, max_length=512, return_tensors='pt')

    # To cuda
    new_encodings = {key: tensor.to(device) for key, tensor in new_encodings.items()}
    model.to(device)

    with torch.no_grad():
        outputs = model(**new_encodings)
        logits = outputs.logits  # Raw predictions (logits)
        # Convert logits to predicted class labels
        predicted_class_ids = torch.argmax(logits, dim=1).to('cpu')
        # Optionally, convert class IDs back to class names
        predicted_labels = label_encoder.inverse_transform(predicted_class_ids.numpy())
    return predicted_labels

# Example usage
prediction = predict("Today, the senator gave a bad briefing to the press.")
print(f"Predicted bias: {prediction}")

Predicted bias: ['Biased']


In [15]:
# Perform predictions on the test dataset
predictions, label_ids, metrics = trainer.predict(test_dataset)

# Convert predictions (logits) to predicted class labels if needed
predicted_class_ids = np.argmax(predictions, axis=1)

# Optionally, if you have a label encoder and want to convert the class IDs back to class names
predicted_labels = label_encoder.inverse_transform(predicted_class_ids)

# Print out the metrics, such as accuracy, precision, recall, etc.
print(metrics)

# Optionally, print out the predictions and true labels for comparison
print("Predictions:", predicted_class_ids)
print("True Labels:", label_ids)

<ipython-input-8-bd375230bbef>:10: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}


{'test_loss': 0.6541956663131714, 'test_runtime': 11.1445, 'test_samples_per_second': 318.99, 'test_steps_per_second': 10.05}
Predictions: [1 0 0 ... 1 0 0]
True Labels: [0 1 0 ... 1 1 0]


In [21]:
# 0 = biased , 1 = non-biased

# ANSI escape codes for colors
RED = "\033[91m"
GREEN = "\033[92m"
RESET = "\033[0m"

correct = 0
to_test = test_texts

for i in range(len(to_test)):

  # Example with conditional coloring
  true_label = "Biased" if label_ids[i] == 0 else "Non-Biased"
  predicted_label = "Biased" if predicted_class_ids[i] == 0 else "Non-Biased"
  # Color red if the labels don't match, green if they do
  color = GREEN if true_label == predicted_label else RED

  print(f"{test_texts[i]}")
  print(f"Predict: {color}{predicted_label}{RESET} | True: {color}{true_label}{RESET}\n\n")

  # Correct counter
  if label_ids[i] == predicted_class_ids[i]:
    correct += 1

print(f"Accuracy: {correct}/{len(to_test)}")

Streaming output truncated to the last 5000 lines.
Australian Prime Minister Scott Morrison defended his leadership and his government's record on climate change Sunday as milder temperatures brought hope of a respite from wildfires that have ravaged three states, claiming 24 lives and destroying almost 2,000 homes.
Predict: Non-Biased | True: Biased


The mother of a one-year-old killed in a Cuban clinic after receiving a routine vaccination launched an awareness campaign Tuesday urging social media users to share a photo of her in the hope that the communist regime gives her answers.
Predict: Non-Biased | True: Non-Biased


Considering the negative, often false propaganda uttered by politicians hostile to Social Security and the financial services industry, and misleading media coverage, that is not surprising, that many young people have been conditioned to think they should not count on Social Security to be there when their time to retire rolls around.
Predict: Biased | True: Bias

In [27]:
print(f"Accuracy: {correct}/{len(to_test)}")
print(f"Percentage: {int( correct/len(to_test)*100 )}/100")

Accuracy: 2398/3555
Percentage: 67/100


In [17]:
# # Zip the results folder
# !zip -r results.zip ./results

# # Download the zipped folder
# from google.colab import files
# files.download('results.zip')